# Darcy equation

In this tutorial we present how to solve an evolutionary Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Let $(0,T)$ with $10=T>0$ be the overall simulation period. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} {q} + \nabla p = {- \rho g \nabla y}\\
p_t + \nabla \cdot {q} = f
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{top} \Omega \times (0,T] \qquad p = \rho g \text{ on } \partial_{bottom} \Omega \times (0,T] \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega \times (0,T] $$
and initial conditions:
$$ p|_{t=0} = (1-y) \rho g \text{ in } \Omega \qquad q|_{t=0} = 0 \text{ in } \Omega $$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
%load_ext Cython

In [2]:
import shutil
import os

import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

from math import ceil

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


### Initial parameters definition

In [3]:
S_s = 0.1
extraction_rate = 0.3

In [4]:
initial_h = lambda x: 1

In [5]:
N = 2
dt = 1

T = 10

output_directory = 'primal_time'

In [6]:
num_steps = ceil(T/dt)+1

### Mesh and $V_h$

In [7]:
key = "flow"

In [8]:
subdomain = pp.StructuredTriangleGrid([N] * 2, [1] * 2)
# convert the grid into a mixed-dimensional grid
pg.convert_from_pp(subdomain)
mdg = pg.as_mdg(subdomain)


In [9]:
h_field = pg.Lagrange1(key)

### BC

#### Dirichlet (essential)

In [10]:
left   = subdomain.nodes[0, :] == 0

dirichlet_flag  = left
dirichlet_value = lambda t: np.array(left, dtype=float)

#### Neumann (natural)

In [11]:
top = subdomain.face_centers[1, :] == 1
right  = subdomain.face_centers[0, :] == 1
bottom = subdomain.face_centers[1, :] == 0

neumann_flag  = np.logical_or(top, np.logical_or(bottom, right))
neumann_value = -h_field.assemble_nat_bc( subdomain, lambda x: extraction_rate, np.where(right)[0])

### Matrix Assembly

In [12]:
%%cython
import numpy as np

def find_ordering(coord: np.array):
    lx = np.argmin(coord[0, :])
    rx = np.argmax(coord[0, :])
    mx = np.setdiff1d(np.array([0,1,2]), np.array([lx, rx]))[0]

    # Vertical Alignment
    if np.abs( coord[0, lx] - coord[0, mx] ) < 1e-7:
        # lx and mx vertical aligned, rx no
        up =   lx if np.argmax(coord[1, np.array([lx, mx])]) == 0 else mx
        down = lx if np.argmin(coord[1, np.array([lx, mx])]) == 0 else mx

        if np.abs( coord[1, up] - coord[1, rx] ) < 1e-7:
            return [up, down, rx]
        else:
            return [down, rx, up]
    else:
        # rx and mx vertical aligned, lx no
        up =   rx if np.argmax(coord[1, np.array([rx, mx])]) == 0 else mx
        down = rx if np.argmin(coord[1, np.array([rx, mx])]) == 0 else mx

        if np.abs( coord[1, up] - coord[1, lx] ) < 1e-7:
            return [up, lx, down]
        else:
            return [down, up, lx]

In [13]:
%%cython
import numpy as np

def q1():
    return np.array([-1, -1])

def q2():
    return np.array([ 1, 0])

def q3():
    return np.array([0,  1])

In [14]:
def local_A(coord):

    element_height = (np.max(coord[1, :]) - np.min(coord[1, :]))
    element_width  = (np.max(coord[0, :]) - np.min(coord[0, :]))

    ordering = find_ordering(coord)

    x0 = coord[:, ordering][:, 0]
    x1 = coord[:, ordering][:, 1]
    x2 = coord[:, ordering][:, 2]
    
    J_T_1_T = np.array([[x2[1]-x0[1], x0[1]-x1[1]],
                        [x0[0]-x2[0], x1[0]-x0[0]]]) / ((x1[0]-x0[0]) * (x2[1]-x0[1]) - (x2[0]-x0[0]) * (x1[1]-x0[1]))
    

    q_funcs = [J_T_1_T @ q1(), J_T_1_T @ q2(), J_T_1_T @ q3()]

    M = np.zeros(shape=(3,3))

    area = element_height * element_width

    for i in range(3):
        for j in range(3):
            M[ ordering[i], ordering[j] ] = q_funcs[j].T @ np.eye(2) @ q_funcs[i] * area

    return M

In [15]:
def stifness():

    # Map the domain to a reference geometry (i.e. equivalent to compute
    # surface coordinates in 1d and 2d)

    _, _, _, _, _, node_coords = pp.map_geometry.map_grid(subdomain)
    faces, cells, sign = sps.find(subdomain.cell_faces)

    # Allocate the data to store matrix entries, that's the most efficient
    # way to create a sparse matrix.
    size = np.power(subdomain.dim + 1, 2) * subdomain.num_cells
    rows_I = np.empty(size, dtype=int)
    cols_J = np.empty(size, dtype=int)
    data_IJ = np.empty(size)
    idx = 0

    cell_nodes = subdomain.cell_nodes()

    for c in np.arange(subdomain.num_cells):
        # For the current cell retrieve its nodes
        loc = slice(cell_nodes.indptr[c], cell_nodes.indptr[c + 1])

        nodes_loc = cell_nodes.indices[loc]
        coord_loc = node_coords[:, nodes_loc]

        # Compute the stiff-H1 local matrix
        A = local_A(coord_loc)

        # Save values for stiff-H1 local matrix in the global structure
        cols = np.tile(nodes_loc, (nodes_loc.size, 1))
        loc_idx = slice(idx, idx + cols.size)
        rows_I[loc_idx] = cols.T.ravel()
        cols_J[loc_idx] = cols.ravel()
        data_IJ[loc_idx] = A.ravel()
        idx += cols.size

    # Construct the global matrices
    return sps.csc_matrix((data_IJ, (rows_I, cols_J)))

In [16]:
M_h = h_field.assemble_mass_matrix(subdomain)

### Solve system

In [17]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [18]:
def save_step(sol, saver, i):
    ins = list()

    ins.append((subdomain, "cell_h", proj_psi @ sol))
    ins.append((subdomain, "cell_p", proj_psi @ (sol - subdomain.nodes[1,:])))

    saver.write_vtu(ins, time_step=i)

In [19]:
sol = [ h_field.interpolate(subdomain, initial_h) ]

In [20]:
# Save the initial solution
saver = pp.Exporter(mdg, 'sol', folder_name=output_directory)
save_step(sol[-1], saver, 0)

NameError: name 'proj_psi' is not defined

In [ ]:
A = h_field.assemble_stiff_matrix(subdomain)
print(A.todense())
spp = S_s / dt * M_h + A

for time_instant_index in range(1, num_steps):
    current_instant = time_instant_index * dt
    rhs = S_s / dt * M_h @ sol[-1] + neumann_value

    ls = pg.LinearSystem(spp, rhs)
    ls.flag_ess_bc(dirichlet_flag, dirichlet_value(current_instant))

    sol.append(ls.solve())
    save_step(sol[-1], saver, time_instant_index)

    print(sol[-1][subdomain.nodes[0, :] == 0])

[[ 1.  -0.5  0.  -0.5  0.   0.   0.   0.   0. ]
 [-0.5  2.  -0.5  0.  -1.   0.   0.   0.   0. ]
 [ 0.  -0.5  1.   0.   0.  -0.5  0.   0.   0. ]
 [-0.5  0.   0.   2.  -1.   0.  -0.5  0.   0. ]
 [ 0.  -1.   0.  -1.   4.  -1.   0.  -1.   0. ]
 [ 0.   0.  -0.5  0.  -1.   2.   0.   0.  -0.5]
 [ 0.   0.   0.  -0.5  0.   0.   1.  -0.5  0. ]
 [ 0.   0.   0.   0.  -1.   0.  -0.5  2.  -0.5]
 [ 0.   0.   0.   0.   0.  -0.5  0.  -0.5  1. ]]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
